In [ ]:
# 시스템 모듈
from dotenv import load_dotenv

# 랭체인 모듈
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

# 환경변수 로드
load_dotenv()

# 백터 DB 설정
DB_INDEX = "MANUAL_DB"
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.load_local(DB_INDEX, embeddings, allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever()


# 챗봇 설정
query = "김장호라는 사람"
res = retriever.invoke(query)

print(len(res))
for item in res:
    print(item.metadata["source"])
    print(len(item.page_content))
    print("------------------------")
